# Spooky author identification

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of movie reviews, accompanied by labels.

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=400px>

Here, we'll pass in words to an embedding layer. We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the word2vec lesson. You can actually train up an embedding with word2vec and use it here. But it's good enough to just have an embedding layer and let the network learn the embedding table on it's own.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. We're using the sigmoid because we're trying to predict if this text has positive or negative sentiment. The output layer will just be a single unit then, with a sigmoid activation function.

We don't care about the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

In [1]:
import numpy as np
import tensorflow as tf
import csv
from sklearn import preprocessing
from string import punctuation

In [79]:
reviews = []
labels = []
test = []
test_ids = []
with open('./text.csv', 'r',encoding="latin-1") as f:
    text_reader = csv.reader(f,delimiter=",")
    next(text_reader)
    for row in text_reader:
        reviews.append(row[0])
with open('./author.csv', 'r') as labels_csv:
    author_reader = csv.reader(labels_csv,delimiter =",")
    next(author_reader) #ignore header
    for row in author_reader:
        labels.append(row[0])
with open("./test.csv",'r') as test_csv:
    text_reader = csv.reader(test_csv, delimiter= ",")
    next(text_reader) #ignore header
    
    for row in text_reader:
        test_ids.append(row[0])
        test.append(row[1])

In [3]:
reviews[:20]

['This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.',
 'It never once occurred to me that the fumbling might be a mere mistake.',
 'In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.',
 'How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.',
 'Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk.',
 'A youth passed in solitude, my best years spent under your gentle and feminine fosterage, h

In [4]:
labels[:20]

['EAP',
 'HPL',
 'EAP',
 'MWS',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'EAP',
 'HPL']

In [57]:
test[:20]

['Still, as I urged our leaving Ireland with such inquietude and impatience, my father thought it best to yield.',
 'If a fire wanted fanning, it could readily be fanned with a newspaper, and as the government grew weaker, I have no doubt that leather and iron acquired durability in proportion, for, in a very short time, there was not a pair of bellows in all Rotterdam that ever stood in need of a stitch or required the assistance of a hammer.',
 'And when they had broken down the frail door they found only this: two cleanly picked human skeletons on the earthen floor, and a number of singular beetles crawling in the shadowy corners.',
 'While I was thinking how I should possibly manage without them, one actually tumbled out of my head, and, rolling down the steep side of the steeple, lodged in the rain gutter which ran along the eaves of the main building.',
 'I am not sure to what limit his knowledge may extend.',
 '"The thick and peculiar mist, or smoke, which distinguishes the Indi

## Data preprocessing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. We'll want to get rid of those periods. Also, you might notice that the reviews are delimited with newlines `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [58]:
from string import punctuation
all_text = ' '.join([c for c in reviews + test if c not in punctuation])

words = all_text.split()

In [59]:
all_text[:305]

'This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall. It never once occurred to me that the fumbling might be a mere mistake. I'

In [60]:
words[:100]

['This',
 'process,',
 'however,',
 'afforded',
 'me',
 'no',
 'means',
 'of',
 'ascertaining',
 'the',
 'dimensions',
 'of',
 'my',
 'dungeon;',
 'as',
 'I',
 'might',
 'make',
 'its',
 'circuit,',
 'and',
 'return',
 'to',
 'the',
 'point',
 'whence',
 'I',
 'set',
 'out,',
 'without',
 'being',
 'aware',
 'of',
 'the',
 'fact;',
 'so',
 'perfectly',
 'uniform',
 'seemed',
 'the',
 'wall.',
 'It',
 'never',
 'once',
 'occurred',
 'to',
 'me',
 'that',
 'the',
 'fumbling',
 'might',
 'be',
 'a',
 'mere',
 'mistake.',
 'In',
 'his',
 'left',
 'hand',
 'was',
 'a',
 'gold',
 'snuff',
 'box,',
 'from',
 'which,',
 'as',
 'he',
 'capered',
 'down',
 'the',
 'hill,',
 'cutting',
 'all',
 'manner',
 'of',
 'fantastic',
 'steps,',
 'he',
 'took',
 'snuff',
 'incessantly',
 'with',
 'an',
 'air',
 'of',
 'the',
 'greatest',
 'possible',
 'self',
 'satisfaction.',
 'How',
 'lovely',
 'is',
 'spring',
 'As',
 'we',
 'looked',
 'from',
 'Windsor']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [10]:
reviews[2]

'In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.'

In [61]:
# Create your dictionary that maps vocab words to integers here
vocab_to_int = {word:index for index,word in enumerate(set(words),1)}
vocab_to_int["<PAD>"] = 0

# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = []
for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
    
test_ints = []
for test_line in test:
    test_ints.append([vocab_to_int[word] for word in test_line.split()])

In [12]:
reviews_ints[1]

[43506,
 35609,
 31009,
 15005,
 46680,
 32278,
 9416,
 30426,
 12313,
 42514,
 27433,
 9969,
 36793,
 31000]

In [63]:
test_ints[1]

[19698,
 11909,
 51240,
 23965,
 4911,
 45488,
 17821,
 36777,
 32738,
 8824,
 45154,
 11909,
 48164,
 18567,
 51493,
 36288,
 13742,
 25748,
 18494,
 40983,
 12773,
 7272,
 17896,
 11270,
 49566,
 18567,
 32802,
 42820,
 23099,
 33032,
 11790,
 46783,
 33032,
 11909,
 3187,
 20679,
 16896,
 25196,
 8892,
 3960,
 11909,
 40378,
 35021,
 13695,
 33032,
 35441,
 38057,
 11270,
 48567,
 41609,
 33032,
 14337,
 35021,
 11909,
 33085,
 5466,
 52011,
 36288,
 35137,
 35021,
 11909,
 31410]

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively.

In [13]:
labels_to_int = {}
int_to_labels = {}
unique_labels = list(set(labels))
for i,label in enumerate(unique_labels):
    labels_to_int[label] = i
    int_to_labels[i] = label
    
int_labels = []

for label in labels:
    int_labels.append(labels_to_int[label])
    
print(labels_to_int)
print(int_to_labels)
print(int_labels[:10])

{'EAP': 0, 'MWS': 1, 'HPL': 2}
{0: 'EAP', 1: 'MWS', 2: 'HPL'}
[0, 2, 0, 1, 2, 1, 0, 0, 0, 1]


In [14]:
encoder = preprocessing.LabelBinarizer()
encoder.fit(list(set(int_labels)))
one_hot_labels = encoder.transform(int_labels)
                                   
one_hot_labels

array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       ..., 
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1]])

In [15]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
print("Minimum length: {}".format(min(review_lens)))
print("Average length: {}".format(sum(review_lens)/len(review_lens)))

Zero-length reviews: 0
Maximum review length: 861
Minimum length: 2
Average length: 92.09655172413792


In [64]:
test_lens = Counter([len(x) for x in test_ints])
print("Zero-length reviews: {}".format(test_lens[0]))
print("Maximum review length: {}".format(max(test_lens)))
print("Minimum length: {}".format(min(test_lens)))
print("Average length: {}".format(sum(test_lens)/len(test_lens)))

Zero-length reviews: 0
Maximum review length: 818
Minimum length: 3
Average length: 77.57377049180327


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. Let's truncate to 200 steps. For reviews shorter than 200, we'll pad with 0s. For reviews longer than 200, we can truncate them to the first 200 characters.

> **Exercise:** Now, create an array `features` that contains the data we'll pass to the network. The data should come from `review_ints`, since we want to feed integers to the network. Each row should be 200 elements long. For reviews shorter than 200 words, left pad with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. For reviews longer than 200, use on the first 200 words as the feature vector.

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.



In [16]:
seq_len = 100
features = []

for review in reviews_ints:
    review_size = len(review)
    if review_size < seq_len:
        padded_review = [0] * seq_len
        padded_review[seq_len-len(review):seq_len] = review
    elif review_size > seq_len:
        padded_review = review[:seq_len]
    
    features.append(padded_review)
features  = np.array(features)

In [17]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 16345, 16059,  3477, 33108,
        32278,  6082, 30138, 29371, 31451, 30426, 26129, 29371, 46952,
        45385, 43223, 34351, 42514, 14458,    22, 16893, 15524, 26205,
        46680, 30426, 20027, 40518, 34351, 36846,  5775, 21501, 15160,
        46208, 29371, 30426, 10049,   381, 24125,  1184, 13206, 30426,
        37249],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [70]:
test_features = []

for test_line in test_ints:
    line_size = len(test_line)
    if line_size < seq_len:
        padded_line = [0] * seq_len
        padded_line[seq_len-len(test_line):seq_len] = test_line
    elif line_size > seq_len:
        padded_line = test_line[:seq_len]
        
    test_features.append(padded_line)

test_features = np.array(test_features)

In [71]:
test_features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
        56213, 51493, 40983, 40029, 42308, 34752, 48472, 45154, 34757,
        48182, 18567,  6556, 55949,  3661, 26742, 45488, 23804, 55629,
          606],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets here. You'll need to create sets for the features and the labels, `train_x` and `train_y` for example. Define a split fraction, `split_frac` as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9. The rest of the data will be split in half to create the validation and testing data.

In [18]:
split_frac = 0.8

split_index  = int(len(features)*split_frac)

train_x, val_x = features[:split_index],features[split_index:]
train_y, val_y = one_hot_labels[:split_index],one_hot_labels[split_index:]

split_index = int(len(val_x)/2)

val_x, test_x = val_x[:split_index],val_x[split_index:]
val_y, test_y = val_y[:split_index],val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))


			Feature Shapes:
Train set: 		(15663, 100) 
Validation set: 	(1958, 100) 
Test set: 		(1958, 100)


With train, validation, and text fractions of 0.8, 0.1, 0.1, the final shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

In [19]:
lstm_size = 512
lstm_layers = 1
batch_size = 500
learning_rate = 0.0005
dropout_prob = 0.10

For the network itself, we'll be passing in our 200 element long review vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

> **Exercise:** Create the `inputs_`, `labels_`, and drop out `keep_prob` placeholders using `tf.placeholder`. `labels_` needs to be two-dimensional to work with some functions later.  Since `keep_prob` is a scalar (a 0-dimensional tensor), you shouldn't provide a size to `tf.placeholder`.

In [20]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32,shape=[batch_size,None],name="inputs")
    labels_ = tf.placeholder(tf.int32,shape=[batch_size,len(unique_labels)],name = "labels")
    keep_prob = tf.placeholder(tf.float32,name ="keep_prob")

### Embedding

Now we'll add an embedding layer. We need to do this because there are 74000 words in our vocabulary. It is massively inefficient to one-hot encode our classes here. You should remember dealing with this problem from the word2vec lesson. Instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using word2vec, then load it here. But, it's fine to just make a new layer and let the network learn the weights.

> **Exercise:** Create the embedding lookup matrix as a `tf.Variable`. Use that embedding matrix to get the embedded vectors to pass to the LSTM cell with [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup). This function takes the embedding matrix and an input tensor, such as the review vectors. Then, it'll return another tensor with the embedded vectors. So, if the embedding layer has 200 units, the function will return a tensor with size [batch_size, 200].



In [21]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words,embed_size),-0.5,0.5))
    embed = tf.nn.embedding_lookup(embedding,inputs_)

### LSTM cell

<img src="assets/network_diagram.png" width=400px>

Next, we'll create our LSTM cells to use in the recurrent network ([TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)). Here we are just defining what the cells look like. This isn't actually building the graph, just defining the type of cells we want in our graph.

To create a basic LSTM cell for the graph, you'll want to use `tf.contrib.rnn.BasicLSTMCell`. Looking at the function documentation:

```
tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0, input_size=None, state_is_tuple=True, activation=<function tanh at 0x109f1ef28>)
```

you can see it takes a parameter called `num_units`, the number of units in the cell, called `lstm_size` in this code. So then, you can write something like 

```
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

to create an LSTM cell with `num_units`. Next, you can add dropout to the cell with `tf.contrib.rnn.DropoutWrapper`. This just wraps the cell in another cell, but with dropout added to the inputs and/or outputs. It's a really convenient way to make your network better with almost no effort! So you'd do something like

```
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
```

Most of the time, your network will have better performance with more layers. That's sort of the magic of deep learning, adding more layers allows the network to learn really complex relationships. Again, there is a simple way to create multiple layers of LSTM cells with `tf.contrib.rnn.MultiRNNCell`:

```
cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
```

Here, `[drop] * lstm_layers` creates a list of cells (`drop`) that is `lstm_layers` long. The `MultiRNNCell` wrapper builds this into multiple layers of RNN cells, one for each cell in the list.

So the final cell you're using in the network is actually multiple (or just one) LSTM cells with dropout. But it all works the same from an achitectural viewpoint, just a more complicated graph in the cell.

> **Exercise:** Below, use `tf.contrib.rnn.BasicLSTMCell` to create an LSTM cell. Then, add drop out to it with `tf.contrib.rnn.DropoutWrapper`. Finally, create multiple LSTM layers with `tf.contrib.rnn.MultiRNNCell`.

Here is [a tutorial on building RNNs](https://www.tensorflow.org/tutorials/recurrent) that will help you out.


In [22]:
with graph.as_default():
    # Your basic LSTM cell
    #lstm = tf.contrib.rnn.BasicLSTMCell(num_units=lstm_size)
    
    # Add dropout to the cell
    #drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    #cell = tf.contrib.rnn.MultiRNNCell([drop]*lstm_layers)
    cell_list = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(num_units=lstm_size) ,output_keep_prob=keep_prob)  ]
    cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

### RNN forward pass

<img src="assets/network_diagram.png" width=400px>

Now we need to actually run the data through the RNN nodes. You can use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) to do this. You'd pass in the RNN cell you created (our multiple layered LSTM `cell` for instance), and the inputs to the network.

```
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
```

Above I created an initial state, `initial_state`, to pass to the RNN. This is the cell state that is passed between the hidden layers in successive time steps. `tf.nn.dynamic_rnn` takes care of most of the work for us. We pass in our cell and the input to the cell, then it does the unrolling and everything else for us. It returns outputs for each time step and the final_state of the hidden layer.

> **Exercise:** Use `tf.nn.dynamic_rnn` to add the forward pass through the RNN. Remember that we're actually passing in vectors from the embedding layer, `embed`.



In [23]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell,embed,initial_state=initial_state)

### Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with `outputs[:, -1]`, the calculate the cost from that and `labels_`.

In [24]:
with graph.as_default():
    fully_connected = tf.contrib.layers.fully_connected(outputs[:, -1], 3, activation_fn=tf.nn.relu)
    logits = tf.identity(fully_connected)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=labels_))
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

In [30]:
with graph.as_default():
    predictions = tf.nn.softmax(logits)
#    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
#    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [26]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [27]:
for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
    print(y)

[[1 0 0]
 [0 0 1]
 [1 0 0]
 ..., 
 [0 1 0]
 [0 1 0]
 [0 0 1]]
[[1 0 0]
 [0 1 0]
 [1 0 0]
 ..., 
 [1 0 0]
 [0 1 0]
 [0 0 1]]
[[0 0 1]
 [0 0 1]
 [0 0 1]
 ..., 
 [1 0 0]
 [0 0 1]
 [0 0 1]]
[[0 1 0]
 [0 0 1]
 [0 0 1]
 ..., 
 [0 0 1]
 [1 0 0]
 [1 0 0]]
[[1 0 0]
 [0 0 1]
 [0 0 1]
 ..., 
 [0 0 1]
 [1 0 0]
 [0 1 0]]
[[0 1 0]
 [0 0 1]
 [1 0 0]
 ..., 
 [0 1 0]
 [0 1 0]
 [0 0 1]]
[[0 1 0]
 [1 0 0]
 [1 0 0]
 ..., 
 [1 0 0]
 [1 0 0]
 [1 0 0]]
[[1 0 0]
 [1 0 0]
 [0 1 0]
 ..., 
 [0 0 1]
 [0 1 0]
 [0 0 1]]
[[0 1 0]
 [1 0 0]
 [1 0 0]
 ..., 
 [1 0 0]
 [0 0 1]
 [1 0 0]]
[[0 0 1]
 [1 0 0]
 [0 0 1]
 ..., 
 [0 1 0]
 [0 0 1]
 [1 0 0]]
[[0 0 1]
 [1 0 0]
 [1 0 0]
 ..., 
 [1 0 0]
 [0 0 1]
 [1 0 0]]
[[1 0 0]
 [1 0 0]
 [1 0 0]
 ..., 
 [0 0 1]
 [0 0 1]
 [0 1 0]]
[[1 0 0]
 [1 0 0]
 [0 0 1]
 ..., 
 [1 0 0]
 [1 0 0]
 [0 1 0]]
[[0 0 1]
 [1 0 0]
 [0 0 1]
 ..., 
 [1 0 0]
 [0 0 1]
 [1 0 0]]
[[0 1 0]
 [0 0 1]
 [0 1 0]
 ..., 
 [1 0 0]
 [1 0 0]
 [0 0 1]]
[[1 0 0]
 [0 0 1]
 [0 0 1]
 ..., 
 [0 0 1]
 [1 0 0]
 [1 0 0]]
[[1 0 0]

## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. Before you run this, make sure the `checkpoints` directory exists.

In [38]:
for x, y in get_batches(val_x, val_y, batch_size):
    print(y, np.argmax(y,1))

[[1 0 0]
 [1 0 0]
 [0 0 1]
 ..., 
 [1 0 0]
 [0 1 0]
 [1 0 0]] [0 0 2 0 2 2 0 0 0 1 1 2 2 0 0 2 1 0 0 2 1 1 0 0 0 2 0 1 0 2 0 0 1 2 0 1 0
 1 0 1 1 0 1 0 0 2 0 0 2 2 2 0 2 0 2 0 0 0 2 0 1 1 2 2 1 2 1 2 2 2 0 1 0 2
 2 0 1 1 0 2 2 0 0 2 2 1 2 1 1 2 1 0 2 0 0 0 0 0 1 0 0 2 1 2 0 0 0 1 0 2 1
 0 0 2 1 0 0 0 0 2 0 0 2 1 2 2 0 1 1 0 0 1 0 0 1 2 2 0 2 2 0 0 2 2 0 1 1 0
 1 0 2 2 2 2 0 2 2 1 0 1 0 2 0 0 1 1 0 2 0 1 0 1 0 1 0 0 2 0 0 1 2 0 0 0 0
 1 0 0 0 1 1 0 2 2 1 1 0 0 0 0 2 0 2 0 1 2 1 0 0 0 0 2 0 0 1 2 0 1 2 0 0 2
 1 1 2 0 2 1 1 2 1 0 2 1 0 2 2 0 1 1 2 0 1 2 1 0 1 1 1 0 2 0 2 2 2 0 0 2 0
 2 1 0 0 2 0 1 2 1 2 0 0 0 2 0 2 2 1 0 2 1 1 0 0 2 1 0 0 0 1 1 1 1 2 1 1 2
 1 1 2 1 1 0 0 2 2 2 0 0 1 2 1 1 2 1 0 1 0 1 0 0 1 2 1 1 0 1 2 0 2 0 1 0 1
 0 1 0 2 1 1 0 2 0 0 2 1 2 0 1 2 0 2 0 2 0 1 2 1 1 2 0 1 1 2 0 1 2 1 1 2 0
 0 1 1 0 0 2 2 0 0 2 2 1 0 0 0 1 0 2 2 1 0 0 2 2 2 1 2 0 0 0 2 0 1 1 0 2 0
 0 0 0 0 2 1 2 2 0 1 1 0 0 2 2 0 2 0 2 0 2 1 0 1 0 2 2 2 2 1 1 2 0 1 1 0 0
 1 1 2 1 2 1 0 1 1 0 0 2 2 1 2 0 1 2 1

array([1, 2])

In [45]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y,
                    keep_prob: dropout_prob,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed_dict = {inputs_: x,
                            labels_: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    
                    val_prediction = sess.run(predictions,feed_dict=feed_dict)
                    val_cost = sess.run(cost,feed_dict=feed_dict)
                    val_prediction_hardmax = np.argmax(val_prediction,1)
                    val_real_hardmax = np.argmax(y,1)
                    val_accuracy = np.sum(val_real_hardmax == val_prediction_hardmax) / y.shape[0]
                    print("val cost",val_cost , val_accuracy )
                    
                    #batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    #val_acc.append(batch_acc)
                #print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/checkpoint.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 1.055
Epoch: 0/10 Iteration: 10 Train loss: 1.082
Epoch: 0/10 Iteration: 15 Train loss: 1.087
Epoch: 0/10 Iteration: 20 Train loss: 1.073
Epoch: 0/10 Iteration: 25 Train loss: 1.026
val cost 1.03584 0.482
val cost 1.04058 0.49
val cost 1.05697 0.442
Epoch: 0/10 Iteration: 30 Train loss: 1.071
Epoch: 1/10 Iteration: 35 Train loss: 0.997
Epoch: 1/10 Iteration: 40 Train loss: 1.002
Epoch: 1/10 Iteration: 45 Train loss: 0.942
Epoch: 1/10 Iteration: 50 Train loss: 1.021
val cost 0.960069 0.516
val cost 0.958095 0.542
val cost 0.979355 0.526
Epoch: 1/10 Iteration: 55 Train loss: 0.979
Epoch: 1/10 Iteration: 60 Train loss: 0.981
Epoch: 2/10 Iteration: 65 Train loss: 0.902
Epoch: 2/10 Iteration: 70 Train loss: 0.928
Epoch: 2/10 Iteration: 75 Train loss: 0.899
val cost 0.860533 0.614
val cost 0.866047 0.628
val cost 0.894679 0.606
Epoch: 2/10 Iteration: 80 Train loss: 0.832
Epoch: 2/10 Iteration: 85 Train loss: 0.798
Epoch: 2/10 Iteration: 90 Train loss: 0.7

## Testing

In [46]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        
        test_prediction = sess.run(predictions,feed_dict=feed_dict)
        test_cost = sess.run(cost,feed_dict=feed_dict)
        test_prediction_hardmax = np.argmax(test_prediction,1)
        test_real_hardmax = np.argmax(y,1)
        test_accuracy = np.sum(val_real_hardmax == test_prediction_hardmax) / y.shape[0]
        print("test cost",test_cost , test_accuracy )
        #batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        #test_acc.append(batch_acc)
        #print(x.shape,y.shape)
    #print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt
test cost 0.796144 0.748
test cost 0.796144 0.748
test cost 0.796144 0.748


In [52]:
for prediction in  test_prediction_hardmax :
    print(int_to_labels[prediction])

EAP
MWS
MWS
EAP
EAP
EAP
MWS
MWS
HPL
HPL
HPL
HPL
HPL
MWS
MWS
EAP
HPL
HPL
MWS
EAP
HPL
MWS
HPL
HPL
HPL
HPL
MWS
MWS
HPL
MWS
EAP
MWS
HPL
HPL
MWS
EAP
HPL
HPL
HPL
EAP
MWS
EAP
EAP
EAP
HPL
EAP
MWS
MWS
MWS
MWS
EAP
HPL
MWS
EAP
MWS
HPL
HPL
MWS
EAP
MWS
MWS
MWS
MWS
MWS
MWS
MWS
EAP
EAP
HPL
HPL
EAP
HPL
MWS
MWS
MWS
MWS
EAP
HPL
EAP
EAP
EAP
MWS
HPL
MWS
HPL
MWS
HPL
EAP
EAP
EAP
MWS
HPL
HPL
HPL
MWS
EAP
MWS
EAP
EAP
HPL
EAP
MWS
HPL
HPL
HPL
HPL
EAP
EAP
HPL
MWS
HPL
EAP
HPL
MWS
MWS
EAP
EAP
EAP
MWS
HPL
MWS
MWS
HPL
EAP
MWS
HPL
MWS
EAP
EAP
EAP
EAP
EAP
MWS
HPL
EAP
MWS
HPL
MWS
MWS
MWS
EAP
EAP
MWS
MWS
HPL
EAP
MWS
EAP
EAP
MWS
EAP
MWS
EAP
HPL
EAP
HPL
MWS
EAP
HPL
MWS
MWS
EAP
HPL
MWS
MWS
MWS
MWS
HPL
HPL
EAP
EAP
EAP
HPL
HPL
MWS
MWS
HPL
HPL
EAP
EAP
EAP
MWS
EAP
EAP
MWS
HPL
MWS
HPL
MWS
EAP
EAP
EAP
MWS
EAP
EAP
EAP
MWS
MWS
MWS
HPL
MWS
EAP
MWS
HPL
MWS
EAP
HPL
EAP
EAP
HPL
EAP
EAP
HPL
EAP
EAP
EAP
HPL
EAP
EAP
MWS
HPL
EAP
HPL
HPL
EAP
MWS
EAP
EAP
HPL
EAP
EAP
EAP
EAP
HPL
EAP
EAP
HPL
MWS
MWS
HPL
MWS
EAP
MWS
EAP
HPL
HPL
MWS
EAP
EAP
MWS


In [90]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    
    feed = {inputs_: test_features,
                keep_prob: 1,
                initial_state: test_state}
    
    test_prediction = sess.run(predictions,feed_dict=feed_dict)
    test_prediction_hardmax = np.argmax(test_prediction,1)
    
    submit_file = open("submission.csv","w")
    csv_writer = csv.writer(submit_file,delimiter = ",")
    
    for i,prediction in  enumerate(test_prediction_hardmax) :
        EAP_prob = test_prediction[i,labels_to_int["EAP"]]
        HPL_prob = test_prediction[i,labels_to_int["HPL"]]
        MWS_prob = test_prediction[i,labels_to_int["MWS"]]
        line = [test_ids[i] , EAP_prob, HPL_prob, MWS_prob]
        print(test_ids[i] , " ", int_to_labels[prediction] , EAP_prob, HPL_prob, MWS_prob)
        csv_writer.writerow(line)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt
id02310   EAP 0.67175 0.0413983 0.286852
id24541   MWS 0.0032757 0.0032757 0.993449
id00134   MWS 0.000147423 0.000147423 0.999705
id27757   EAP 0.99905 0.000475034 0.000475034
id04081   EAP 0.991923 0.00403875 0.00403875
id27337   EAP 0.962245 0.0188774 0.0188774
id24265   MWS 0.000658416 0.000658416 0.998683
id25917   MWS 3.55025e-05 3.55025e-05 0.999929
id04951   HPL 0.00155577 0.996888 0.00155577
id14549   HPL 0.0303536 0.939293 0.0303536
id22505   HPL 0.04753 0.90494 0.04753
id24002   HPL 0.0177487 0.964503 0.0177487
id18982   HPL 0.00125326 0.997494 0.00125326
id15181   MWS 0.144808 0.144808 0.710383
id21888   MWS 0.100812 0.044849 0.854339
id12035   EAP 0.995381 0.0023094 0.0023094
id17991   HPL 0.000824047 0.998352 0.000824047
id10707   HPL 0.0034821 0.993036 0.0034821
id07101   MWS 0.210072 0.347473 0.442455
id00345   EAP 0.901451 0.0492745 0.0492745
id05912   HPL 0.00411429 0.991771 0.00411429
id13443   MWS